### Maximize the loss with no targets

In [1]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset

import advertorch
from advertorch.attacks import GradientSignAttack
from advertorch.attacks import LinfPGDAttack
from advertorch.attacks import CarliniWagnerL2Attack

# Need the model for which AE is generated here
from Resnet_adaptation import new_cnn_model

In [2]:
# RANDOMIZE
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Generate AEs for the second model
PATH = './ha/best_model_CNN.pt'
model = torch.load(PATH).to(device) if torch.cuda.is_available() else torch.load(PATH, map_location=torch.device('cpu'))

In [5]:
# get scalers
training_set = np.load('./data/train.npz')

mean = round(training_set['x'][...,0].mean(), 2)
std = round(training_set['x'][...,0].std(), 2)

print("Mean ={}, Std = {}".format(mean, std))

Mean =4090.28, Std = 2164.09


In [6]:
# How many? Due to computational limitation only 320, max 2616
#num_aes = 2616

# randomly select 320?


In [7]:
# data for which the AEs are to be generated 
test_inputs = np.load('./ha/test_inputs.npy', allow_pickle = True)
print(f'shape = {test_inputs.shape}, Max = {np.max(test_inputs)}')

test_inputs = (test_inputs - mean)/std
print(f'shape = {test_inputs.shape}, Max = {np.max(test_inputs)}')

# Use the max and min values to constain the AEs in the box
clip_min = np.min(test_inputs)
clip_max = np.max(test_inputs)

print(f'Box Constraints {clip_min},{clip_max}')

shape = (2609, 12, 150), Max = 11644.0
shape = (2609, 12, 150), Max = 3.4904832978295723
Box Constraints -1.8900692669898203,3.4904832978295723


In [8]:
class TR_dataset_no_targets(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, key):
        input_seq = self.data_list[key]
        return input_seq

In [9]:
BATCH_SIZE = 1
adv_set = TR_dataset_no_targets(test_inputs)

adv_loader = torch.utils.data.DataLoader(dataset=adv_set,
                                               batch_size=BATCH_SIZE,
                                               collate_fn=None,
                                               shuffle=False)

In [10]:
# same rmse loss fn, in PyTorch
def loss_fun(out, tar):
    rmse = torch.sqrt(torch.mean((out - tar)**2))
    return rmse

### FGSM

In [11]:
def generate_fgsm_no_target(model, loss_func, eps, adv_dataloader, cmin, cmax):
    
    FGSM_adv_signals =[]
    eps_value = torch.tensor([eps], dtype = torch.float32).to(device)
    
    adversary = GradientSignAttack(model, 
                                   loss_fn= loss_func,
                                   eps = eps_value, 
                                   clip_min=cmin, 
                                   clip_max=cmax, 
                                   targeted= False)
    
    for bi, data in enumerate(adv_dataloader):
        data_batch = data.float().to(device)
        adv_signal = adversary.perturb(data_batch)
        FGSM_adv_signals.append(np.squeeze(adv_signal.cpu().numpy()))
        
    return FGSM_adv_signals

In [12]:
epsillon = [0., 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.2]

print(epsillon[7])

adv_fgsm = generate_fgsm_no_target(model, loss_fun, epsillon[7], adv_loader, clip_min, clip_max )
adv_fgsm= np.array(adv_fgsm)

print(adv_fgsm.shape)
np.save('./ha/adv_fgsm.npy', adv_fgsm)

0.2
(2609, 12, 150)


### BIM

In [13]:
def generate_bim_no_target(model, loss_func, eps, adv_dataloader, cmin, cmax):
    
    BIM_adv_signals =[]
    eps_value = torch.tensor([eps], dtype = torch.float32).to(device)
    
    adversary = LinfPGDAttack(model, 
                              loss_fn= loss_func,
                              eps = eps_value,
                              eps_iter=0.05, # per iteration change
                              nb_iter=10, #num_iterations
                              clip_min=cmin, 
                              clip_max=cmax, 
                              targeted= False)
    
    for bi, data in enumerate(adv_dataloader):
        data_batch = data.float().to(device)
        adv_signal = adversary.perturb(data_batch)
        BIM_adv_signals.append(np.squeeze(adv_signal.cpu().numpy()))
        
    return BIM_adv_signals

In [14]:
print(epsillon[7])

adv_bim = generate_bim_no_target(model, loss_fun, epsillon[7], adv_loader, clip_min, clip_max )
adv_bim= np.array(adv_bim)

print(adv_bim.shape)
np.save('./ha/adv_bim.npy', adv_bim)

0.2
(2609, 12, 150)


In [30]:
# common metric
# define a common metric
def measure_l2(one, two):
    # Both will be 150*12 network wide snapshots
    one = np.squeeze(np.array(one))
    two = np.squeeze(np.array(two))
    #print(one.shape)
    #print(two.shape)
    #c = one - two
    #print("C",c.shape)
    l2 = np.sum(np.square(one-two))
    return l2

def notation(number):
    str_num = str(number)
    splits = str_num.split('.')
    leng = int(len(splits[0]))-1
    divisor = 1
    for i in range(leng):
        divisor = divisor*10
    changed_num = round((number/divisor),2)
    return changed_num, leng

In [31]:
originals = np.squeeze(np.load('./data/test.npz')['x'])[0:2609]
print(np.max(originals), originals.shape)

fgsm_flow = np.load('./ha/adv_fgsm.npy', allow_pickle = True)
print(np.max(fgsm_flow), fgsm_flow.shape)

bim_flow = np.load('./ha/adv_bim.npy', allow_pickle = True)
print(np.max(bim_flow), bim_flow.shape)

fgsm_flow = (fgsm_flow*std) + mean
print(np.max(fgsm_flow), fgsm_flow.shape)

bim_flow = (bim_flow*std) + mean
print(np.max(bim_flow), bim_flow.shape)

11644.0 (2609, 12, 150)
3.4904833 (2609, 12, 150)
3.4904833 (2609, 12, 150)
11644.0 (2609, 12, 150)
11644.0 (2609, 12, 150)


In [32]:
# How much signal changed? common metric

# change in signal
fgsm_sig_change_collect = []
for i in range(len(originals)):
    sig_change = measure_l2(originals[i], fgsm_flow[i])
    fgsm_sig_change_collect.append(sig_change)
fgsm_sig_change_collect = np.array(fgsm_sig_change_collect)
print(f'Signal: shape = {fgsm_sig_change_collect.shape}, max = {np.max(fgsm_sig_change_collect)}' )

fgsm_mean = np.mean(fgsm_sig_change_collect)
num_fgsm, exp_fgsm = notation(fgsm_mean)
print("FGSM: Mean = {}, a.k.a {}E{}".format(fgsm_mean,num_fgsm, exp_fgsm) )

Signal: shape = (2609,), max = 27942247630.585083
FGSM: Mean = 13191997674.486603, a.k.a 1.32E10


In [33]:
# change in signal
bim_sig_change_collect = []
for i in range(len(originals)):
    sig_change = measure_l2(originals[i], bim_flow[i])
    bim_sig_change_collect.append(sig_change)
bim_sig_change_collect = np.array(bim_sig_change_collect)
print(f'Signal: shape = {bim_sig_change_collect.shape}, max = {np.max(bim_sig_change_collect)}' )

bim_mean = np.mean(bim_sig_change_collect)
num_bim, exp_bim = notation(bim_mean)
print("FGSM: Mean = {}, a.k.a {}E{}".format(bim_mean,num_bim, exp_bim) )

Signal: shape = (2609,), max = 26359225257.269608
FGSM: Mean = 12910686260.948496, a.k.a 1.29E10


In [ ]:
# we know preds dont change

### Get ResNet predictions on AEs

- For now, I directly move over to GC-GRNN. Not caring much about ResNet

In [ ]:
# For ResNet: Input, Corresponding AEs, Ground Truth, Prediction on input, Prediction on AE 

In [ ]:
# RMSE Error distribution on test inputs, on AEs